In [ ]:
# Importation des librairies
from my_function import ( 
    sel_classif_pixel,
    report_from_dict_to_df,
    supprimer_dossier_non_vide)
import os
import geopandas as gpd
import numpy as np
import pandas as pd
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, \
    accuracy_score
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw
import classification as cla
import plots

In [6]:
# Création du dossier de sauvegarde temporaire et paramètres
output_dir = "/home/onyxia/work/output_classif"
os.makedirs(output_dir, exist_ok=True) # Création d'un dossier output temporaire

sample_path = "/home/onyxia/work/Depot_Git/results/data/sample/Sample_BD_foret_T31TCJ.shp"
path_sample_px = os.path.join(output_dir,"sample_classif_px.shp")

sample_rasterized = os.path.join(output_dir,"rasterized_sample.tif")
path_image_3b = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_3_bands.tif"
path_image_allbands = "/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif"

path_sample_px_centroid = os.path.join(output_dir,"sample_px_centroid.shp")

In [ ]:
# Save d'un vecteur echantillons avec que les données pour classif pixel
sample = gpd.read_file(sample_path)
sample_px = sel_classif_pixel(sample[['Code',"geometry"]])
sample_px.to_file(path_sample_px)

In [ ]:
# cmd rasterisation paramètres
in_vector = os.path.join(output_dir,'sample_classif_px.shp')
ref_image = '/home/onyxia/work/Depot_Git/results/data/img_pretraitees/Serie_temp_S2_allbands.tif'
ref_image_open = rw.open_image(ref_image)
out_image = os.path.join(output_dir, "rasterized_sample.tif")
field_name = 'Code'  # field containing the numeric label of the classes
sptial_resolution = rw.get_pixel_size(ref_image_open)[0]
xmin,ymax = rw.get_origin_coordinates(ref_image_open)
y,x = rw.get_image_dimension(ref_image_open)[0:2]
xmax,ymin = xmin+x*10,ymax-y*10

# define command pattern to fill with paremeters
cmd_pattern = ("gdal_rasterize -a {field_name} "
               "-tr {sptial_resolution} {sptial_resolution} "
               "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
               "{in_vector} {out_image}")

# fill the string with the parameter thanks to format function
cmd = cmd_pattern.format(in_vector=in_vector, xmin=xmin, ymin=ymin, xmax=xmax,
                         ymax=ymax, out_image=out_image, field_name=field_name,
                         sptial_resolution=sptial_resolution)

# execute the command in the terminal
os.system(cmd)

In [ ]:
# Extraction des échantillons à partir d'un fichier de points pour tenir compte de l'appartenance aux polygones

sample_px['geom_points'] = sample_px.centroid                  # Calcul des centroides
sample_px_centroid = sample_px[['Code','geom_points']]
sample_px_centroid.to_file(path_sample_px_centroid)      # Save des centroides dans le dossier output_classif

# Récupération des coordonnées X
list_row, list_col = rw.get_row_col_from_file (path_sample_px_centroid,path_image_allbands)     # Coordonnées images des points du fichiers points
image_3b = rw.load_img_as_array(path_image_3b)
X = image_3b[(list_row, list_col)]

# Récupération des coordonnées Y
gdf = gpd.read_file(path_sample_px_centroid)             
Y = gdf.loc[:, "Code"].values
Y = np.atleast_2d(Y).T

# Séparation des données avec un jeu train et test
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.7)

In [ ]:
# Application validation statifiée avec appartenance à polygones

list_cm = []
list_accuracy = []
list_report = []

# Iter on stratified K fold
kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=30)
for train, test in kf.split(X, Y):
    X_train, X_test = X[train], X[test]
    Y_train, Y_test = Y[train], Y[test]

    # Modèle Random Forest Classifier
    rfc = RandomForestClassifier(
        max_depth = 50,
        oob_score = True,
        max_samples = 0.75,
        class_weight = 'balanced',
        n_jobs = -1
    )
    rfc.fit(X_train, Y_train)

    Y_predict = rfc.predict(X_test)

    # compute quality
    list_cm.append(confusion_matrix(Y_test, Y_predict))
    list_accuracy.append(accuracy_score(Y_test, Y_predict))
    report = classification_report(Y_test, Y_predict, labels=np.unique(Y_predict), output_dict=True)

    # store them
    list_report.append(report_from_dict_to_df(report))

In [ ]:
## Entraîner le modèle

# Modèle Random Forest Classifier
rfc = RandomForestClassifier(
    max_depth = 50,
    oob_score = True,
    max_samples = 0.75,
    class_weight = 'balanced',
    n_jobs = -1
)

# Fit avec nos valeurs
rfc.fit(X_train,Y_train)

In [ ]:
# Sauvegarder la carte
save_classification(model, images, 'results/data/classif/carte_essences_echelle_pixel.tif')

In [ ]:
## Nettoyage des dossiers
supprimer_dossier_non_vide(output_dir)